In [191]:
import sys
sys.path.append("../")

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from src.functions import (
    get_df_uniques, 
    manage_major_values, 
    get_corr_pairs_thresh
)


In [192]:
# Loads the California housing Dataset
housing = fetch_openml(name="house_prices", as_frame=True)

# Concatenate datas into a single dataframe
data = pd.DataFrame(
    data=np.c_[housing['data'], housing['target']],
    columns=housing['feature_names'] + ['target']
)
data = data.set_index("Id")
data.head()

# Save raw datas
data.to_csv("../data/01/house_prices.csv")

del housing, data

In [193]:
df = pd.read_csv("../data/01/house_prices.csv", index_col="Id") 
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1460 entries, 1.0 to 1460.0
Columns: 80 entries, MSSubClass to target
dtypes: float64(37), object(43)
memory usage: 923.9+ KB


In [194]:
df.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,target
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal,208500.0


In [195]:
nas = df.isna().sum()

print(nas[nas > 0].apply(lambda x: round(x / df.shape[0] * 100, 2)).sort_values(ascending=False))

del nas


PoolQC          99.52
MiscFeature     96.30
Alley           93.77
Fence           80.75
FireplaceQu     47.26
LotFrontage     17.74
GarageType       5.55
GarageYrBlt      5.55
GarageFinish     5.55
GarageQual       5.55
GarageCond       5.55
BsmtExposure     2.60
BsmtFinType2     2.60
BsmtFinType1     2.53
BsmtCond         2.53
BsmtQual         2.53
MasVnrArea       0.55
MasVnrType       0.55
Electrical       0.07
dtype: float64


In [196]:
get_df_uniques(df).sort_values(by=["Uniques count"])

,Features,Dtype,Uniques count,Values
4,Street,object,2,"[Pave, Grvl]"
5,Alley,object,2,"[nan, Grvl, Pave]"
8,Utilities,object,2,"[AllPub, NoSeWa]"
40,CentralAir,object,2,"[Y, N]"
49,HalfBath,float64,3,"[1.0, 0.0, 2.0]"
64,PavedDrive,object,3,"[Y, N, P]"
47,BsmtHalfBath,float64,3,"[0.0, 1.0, 2.0]"
10,LandSlope,object,3,"[Gtl, Mod, Sev]"
71,PoolQC,object,3,"[nan, Ex, Fa, Gd]"
59,GarageFinish,object,3,"[RFn, Unf, Fin, nan]"


In [197]:
df = manage_major_values(df, 90, drop=True)

In [198]:
get_corr_pairs_thresh(df, 20, .7)

0.88248 ('GarageArea', 'GarageCars')
0.82567 ('GarageYrBlt', 'YearBuilt')
0.82549 ('TotRmsAbvGrd', 'GrLivArea')
0.81953 ('TotalBsmtSF', '1stFlrSF')
0.79098 ('target', 'OverallQual')
0.70862 ('target', 'GrLivArea')


In [199]:
df["Garage"] = df["GarageType"].notna()
df['Fence'] = df['Fence'].notna()
df['Basement'] = df["TotalBsmtSF"] > 0
df['2ndFloor'] = df["2ndFlrSF"] > 0

# create the new variable 'age'
df['Age'] = df.apply(lambda x: x['YrSold']-x['YearBuilt'] if (x['YearBuilt'] < x['YearRemodAdd'])
                         else (x['YrSold']-x['YearRemodAdd']), axis=1)

# Delete all df with MSZoning = commercial, agriculture and industrial as these are not residential units
df = df[(df.MSZoning != 'C (all)') & (
    df.MSZoning != 'I (all)') & (df.MSZoning != 'A (agr)')]

# Delete abnormal sales
df = df[(df.SaleCondition != 'Abnorml')]
df = df[(df.SaleCondition != 'Family')]


In [200]:
drop_columns = [
    'GarageType',
    'GarageYrBlt',
    'GarageFinish',
    'GarageCars',
    'GarageArea',
    'GarageQual',
    'YrSold',
    'YearBuilt',
    'YearRemodAdd',
    'TotalBsmtSF',
    '1stFlrSF',
    '2ndFlrSF',
    'BsmtFinSF1',
    'BsmtFinSF2',
    'TotRmsAbvGrd',
    'PoolQC',
    'MiscFeature',
    'Alley'
]

df = df.drop(drop_columns, axis=1)

del drop_columns

In [201]:
get_corr_pairs_thresh(df, 5, .5)

0.79483 ('target', 'OverallQual')
0.70811 ('GrLivArea', 'target')
0.62998 ('GrLivArea', 'FullBath')
0.59422 ('OverallQual', 'GrLivArea')
0.58019 ('2ndFloor', 'HalfBath')


In [202]:
kept_cols = [
    'Age', 
    'GrLivArea', 
    'LotFrontage',
    'LotArea',  
    'Fence', 
    '2ndFloor',
    'Basement',
    'Garage',
    'target'
]

df = df[kept_cols]
cat_cols = df.select_dtypes(include='bool').columns.tolist()
num_cols = df.select_dtypes(exclude='bool').columns.tolist()
df[cat_cols] = df[cat_cols].astype("category")
df[num_cols] = df[num_cols].apply(pd.to_numeric)
df.info()


<class 'pandas.core.frame.DataFrame'>
Float64Index: 1334 entries, 1.0 to 1460.0
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Age          1334 non-null   float64 
 1   GrLivArea    1334 non-null   float64 
 2   LotFrontage  1094 non-null   float64 
 3   LotArea      1334 non-null   float64 
 4   Fence        1334 non-null   category
 5   2ndFloor     1334 non-null   category
 6   Basement     1334 non-null   category
 7   Garage       1334 non-null   category
 8   target       1334 non-null   float64 
dtypes: category(4), float64(5)
memory usage: 68.2 KB


In [205]:
Y = df['target']
X = df.drop('target', axis=1)
print(X.head())

X.corrwith(Y)


      Age  GrLivArea  LotFrontage  LotArea  Fence 2ndFloor Basement Garage
Id                                                                        
1.0   5.0     1710.0         65.0   8450.0  False     True     True   True
2.0  31.0     1262.0         80.0   9600.0  False    False     True   True
3.0   7.0     1786.0         68.0  11250.0  False     True     True   True
5.0   8.0     2198.0         84.0  14260.0  False     True     True   True
6.0  16.0     1362.0         85.0  14115.0   True     True     True   True


Age           -0.518485
GrLivArea      0.708111
LotFrontage    0.346440
LotArea        0.264694
dtype: float64

In [ ]:
df.to_csv("../data/02/cleaned_house_prices.csv")

In [ ]:
df = pd.read_csv("../data/02/cleaned_house_prices.csv", index_col="Id")
df.head()

,Age,GrLivArea,LotFrontage,LotArea,Fence,2ndFloor,Basement,Garage,target
Id,,,,,,,,,
1.0,5.0,1710.0,65.0,8450.0,False,True,True,True,208500.0
2.0,31.0,1262.0,80.0,9600.0,False,False,True,True,181500.0
3.0,7.0,1786.0,68.0,11250.0,False,True,True,True,223500.0
5.0,8.0,2198.0,84.0,14260.0,False,True,True,True,250000.0
6.0,16.0,1362.0,85.0,14115.0,True,True,True,True,143000.0


In [ ]:
# Select Features
X = df[['Age', 'GrLivArea', 'LotFrontage',
          'LotArea', 'GarageArea', 'Fence', 'Pool']]
Y = df[['target']]


KeyError: "['GarageArea', 'Pool'] not in index"

In [ ]:
num_cols = df.select_dtypes(include=[np.number])
cat_cols = df.select_dtypes(include=["O"])


In [ ]:
df.describe(include=['O'])


,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,PavedDrive,SaleType,SaleCondition
count,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1334,1326,1334,1334,1334,1298,1298,1297,1298,1297,1334,1334,1334,1333,1334,1334,724,1334,1334,1334
unique,4,2,4,4,1,5,3,25,9,8,5,8,6,8,14,16,4,4,5,6,4,4,4,6,6,6,5,2,4,4,6,5,3,8,4
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,Y,WD,Normal
freq,1067,1330,837,1201,1334,967,1267,198,1152,1321,1116,660,1037,1311,485,476,781,808,1174,613,584,1197,861,395,1149,1305,694,1254,1227,655,1243,362,1227,1175,1194
